# 01 - Single Agent from Personal Data

This notebook explores creating an LLM persona from personal data and testing how well it captures the essence of the person being modeled.

## Objectives
1. Load personal data (CV, bio, interests, etc.)
2. Create a persona from this data
3. Test the persona with survey questions
4. Evaluate how accurately the persona responds

In [1]:
# Setup - ensure we can import from src
import sys
sys.path.insert(0, '../src')

from centuria.models import Persona, Question, Survey
from centuria.persona import create_persona
from centuria.survey import ask_question, run_survey

## Step 1: Define Personal Context

Replace the context below with your own data. This could include:
- CV/resume summary
- LinkedIn bio
- Personal interests and hobbies
- Reading list / media preferences
- Values and beliefs

In [2]:
# Example context - replace with your own data
# my_context = """
# Name: [Your Name]
# Profession: [Your profession]
# Background: [Brief background]

# Interests:
# - [Interest 1]
# - [Interest 2]

# Values:
# - [Value 1]
# - [Value 2]

# Recent reading/media:
# - [Book/article/show 1]
# - [Book/article/show 2]
# """

# Or load from files:
from centuria.data import load_files
my_context = load_files(['../data/personal/cv.pdf'])

In [3]:
# Create the persona
persona = create_persona(
    name="My Persona",
    context=my_context
)

print(f"Created persona: {persona.name}")
print(f"Context length: {len(persona.context.split())} words")

Created persona: My Persona
Context length: 1149 words


## Step 2: Test with Single Questions

Let's test the persona with individual questions before running a full survey.

In [4]:
# Single select question
q1 = Question(
    id="q1",
    text="Which programming language do you prefer?",
    question_type="single_select",
    options=["Python", "JavaScript", "Rust", "Go"]
)

response = await ask_question(persona, q1)
print(f"Question: {q1.text}")
print(f"Response: {response.response}")

Question: Which programming language do you prefer?
Response: Python


In [5]:
# Open-ended question
q2 = Question(
    id="q2",
    text="What motivates you in your work?",
    question_type="open_ended"
)

response = await ask_question(persona, q2)
print(f"Question: {q2.text}")
print(f"Response: {response.response}")

Question: What motivates you in your work?
Response: I find motivation in the intersection of cutting-edge technology and meaningful societal impact. The opportunity to apply AI and data-driven solutions to tackle complex challenges, whether it's in economic planning or enhancing transparency and democracy, fuels my passion. Collaborating with diverse teams and contributing to projects that drive real-world change keeps me engaged and continuously striving for excellence.


## Step 3: Run a Mini Survey

Test with a small survey to verify the workflow.

In [6]:
# Create a mini survey
mini_survey = Survey(
    id="mini_test",
    name="Mini Test Survey",
    questions=[
        Question(
            id="pref_1",
            text="Do you prefer working alone or in a team?",
            question_type="single_select",
            options=["Alone", "Team", "Mix of both"]
        ),
        Question(
            id="pref_2",
            text="What time of day are you most productive?",
            question_type="single_select",
            options=["Morning", "Afternoon", "Evening", "Night"]
        ),
        Question(
            id="open_1",
            text="Describe your ideal weekend.",
            question_type="open_ended"
        )
    ]
)

# Run the survey
survey_response = await run_survey(persona, mini_survey)

print(f"Survey: {mini_survey.name}")
print(f"Responses:")
for r in survey_response.responses:
    q = next(q for q in mini_survey.questions if q.id == r.question_id)
    print(f"  Q: {q.text}")
    print(f"  A: {r.response}")
    print()

Survey: Mini Test Survey
Responses:
  Q: Do you prefer working alone or in a team?
  A: Mix of both

  Q: What time of day are you most productive?
  A: Morning

  Q: Describe your ideal weekend.
  A: An ideal weekend for me blends relaxation and inspiration. It starts with a leisurely morning coffee while catching up on the latest AI trends or diving into a compelling tech article. I’d spend some time outdoors, perhaps hiking or cycling, to enjoy nature and clear my mind. Evenings would be devoted to creative pursuits, like working on a fun coding project or attending a tech meetup to connect with like-minded people. I’d round off the weekend with good food, maybe trying out a new recipe or dining with friends, and unwinding with a thought-provoking film or book.



## Step 4: Compare with Your Actual Answers

Record your actual answers and compare with the persona's responses.

In [ ]:
# Your actual answers (fill these in)
my_answers = {
    "pref_1": "Mix of both",  # Change to your answer
    "pref_2": "Morning",       # Change to your answer
}

# Compare single-select answers
correct = 0
total = 0

for r in survey_response.responses:
    if r.question_id in my_answers:
        total += 1
        match = r.response == my_answers[r.question_id]
        if match:
            correct += 1
        print(f"{r.question_id}: Persona='{r.response}', Actual='{my_answers[r.question_id]}', Match={match}")

if total > 0:
    accuracy = correct / total
    print(f"\nDistribution Accuracy: {accuracy:.1%} ({correct}/{total})")

## Next Steps

- [ ] Add more personal context data
- [ ] Design the full 50-question validation survey
- [ ] Test different context sizes (500, 1000, 1500 words)
- [ ] Compare different models (GPT-4, Claude)